## What to expect as an Airbnb Host in Berlin

Airbnb has successfully disrupted the traditional hospitality industry as more and more travelers decide to use Airbnb as their primary accommodation provider. Since its inception in 2008, Airbnb has seen an enormous growth, with the number of rentals listed on its website growing exponentially each year.

In Germany, no city is more popular than Berlin. That implies that Berlin is one of the hottest markets for Airbnb in Europe, with over 22,552 listings as of November 2018. With a size of 891 km², this means there are roughly 25 homes being rented out per km² in Berlin on Airbnb!

The following question will drive this project:

> **3. What do visitors like and dislike?**

<br> To find out, we process the reviews to find out what peoples' likes and dislikes are. Natural Language Processing (NLP) and specifically Sentiment Analysis are what we will use here.

### The datasets

I will use the reviews data and combine it with some features from the detailed Berlin listings data, sourced from the Inside Airbnb website. Both datasets were scraped on November 07th 2018.

## > Table of Contents
<a id='Table of contents'></a>

### <a href='#1. Obtaining and Viewing the Data'> 1. Obtaining and Viewing the Data </a>

### <a href='#2. Preprocessing the Data'> 2. Preprocessing the Data </a>
* <a href='#2.1. Dealing with Missing Values'> 2.1. Dealing with Missing Values </a>
* <a href='#2.2. Language Detection'> 2.2. Language Detection </a>

### <a href='#3. Visualizing the Data with WordClouds'> 3. Visualizing the Data with WordClouds </a>

### <a href='#4. Sentiment Analysis'> 4. Sentiment Analysis </a>
* <a href='#4.1. Get used to VADER package'> 4.1. Get used to VADER package </a>
* <a href='#4.2. Calculating Sentiment Scores'> 4.2. Calculating Sentiment Scores </a>
* <a href='#4.3. Comparing Negative and Positive Comments'> 4.3. Comparing Negative and Positive Comments </a>
* <a href='#4.4. Investigating Positive Comments'> 4.4. Investigating Positive Comments </a>
* <a href='#4.5. Investigating Negative Comments'> 4.5. Investigating Negative Comments </a>

### <a href='#5. Appendix'> 5. Appendix </a>

### 1. Obtaining and Viewing the Data 
<a id='1. Obtaining and Viewing the Data'></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

In [ ]:
df_1 = pd.read_csv('../input/berlin-airbnb-data/reviews_summary.csv')
# checking shape ...
print("The dataset has {} rows and {} columns.".format(*df_1.shape))

# ... and duplicates
print("It contains {} duplicates.".format(df_1.duplicated().sum()))

In [ ]:
df_1.head()

Well, it may be valuable to have more details, such as the latitude and longitude of the accommodation that has been reviewed, the neighbourhood it's in, the host id, etc. 

To get this information, let's **combine our reviews_dataframe** with the **listings_dataframe** and take only the columns we need from the latter one:

In [ ]:
df_2 = pd.read_csv('../input/berlin-airbnb-data/listings_summary.csv')
df_2.head()


In [ ]:
# merging full df_1 + add only specific columns from df_2
df = pd.merge(df_1, df_2[['neighbourhood_group_cleansed', 'host_id', 'latitude',
                          'longitude', 'number_of_reviews', 'id', 'property_type']], 
              left_on='listing_id', right_on='id', how='left')

df.rename(columns = {'id_x':'id', 'neighbourhood_group_cleansed':'neighbourhood_group'}, inplace=True)
df.drop(['id_y'], axis=1, inplace=True)

In [ ]:
df.head(3)

In [ ]:
# checking shape
print("The dataset has {} rows and {} columns.".format(*df.shape))

**Hosts with many properties**

By the way, I am curious to find out if any private hosts have started to run a professional business through Airbnb - at least this is what was in the press. Let's work this out:

In [ ]:
# group by hosts and count the number of unique listings --> cast it to a dataframe
properties_per_host = pd.DataFrame(df.groupby('host_id')['listing_id'].nunique())

# sort unique values descending and show the Top20
properties_per_host.sort_values(by=['listing_id'], ascending=False, inplace=True)
properties_per_host.head(20)

Let's take a closer look at the top 3 hosts. How many properties do they have in the different areas? And are these private apartments, or something else, like a hostel?

**> No. 1 Host**

In [ ]:
top1_host = df.host_id == 1625771
df[top1_host].neighbourhood_group.value_counts()

pd.DataFrame(df[top1_host].groupby('neighbourhood_group')['listing_id'].nunique())

In [ ]:
pd.DataFrame(df[top1_host].groupby('property_type')['listing_id'].nunique())

> This host owns apartments in 8 (!) districts. It looks like he was really able to deeply expand a well working business into different neighbourhoods...

**> No. 2 Host**

In [ ]:
top2_host = df.host_id == 8250486
df[top2_host].neighbourhood_group.value_counts()

pd.DataFrame(df[top2_host].groupby('neighbourhood_group')['listing_id'].nunique())

In [ ]:
pd.DataFrame(df[top2_host].groupby('property_type')['listing_id'].nunique())

> Well, looks like the second biggest player turned out to be a hostel.

**> No. 3 Host**

In [ ]:
top3_host = df.host_id == 2293972
df[top3_host].neighbourhood_group.value_counts()

pd.DataFrame(df[top3_host].groupby('neighbourhood_group')['listing_id'].nunique())

In [ ]:
pd.DataFrame(df[top3_host].groupby('property_type')['listing_id'].nunique())

> And host No. 3 also seems to be a professional lodging supplier.

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 2. Preprocessing the Data 
<a id='2. Preprocessing the Data'></a>

#### 2.1. Dealing with Missing Values
<a id='2.1. Dealing with Missing Values'></a>

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

In [ ]:
df.shape

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.2. Language Detection
<a id='2.2. Language Detection'></a>

In [ ]:
# we use Python's langdetect 
from langdetect import detect

In [ ]:
# write the function that detects the language
def language_detection(text):
    try:
        return detect(text)
    except:
        return None

In [ ]:
%%time
df['language'] = df['comments'].apply(language_detection)

In [ ]:
# write the dataframe to a csv file in order to avoid the long runtime
#df.to_csv('../input/processed_df', index=False)
#df = pd.read_csv('../input/processed_df')

In [ ]:
df.language.value_counts().head(10)

In [ ]:
# visualizing the comments' languages a) quick and dirty
ax = df.language.value_counts(normalize=True).head(6).sort_values().plot(kind='barh', figsize=(9,5));

In [ ]:
# visualizing the comments' languages b) neat and clean
ax = df.language.value_counts().head(6).plot(kind='barh', figsize=(9,5), color="lightcoral", 
                                             fontsize=12);

ax.set_title("\nWhat are the most frequent languages comments are written in?\n", 
             fontsize=12, fontweight='bold')
ax.set_xlabel(" Total Number of Comments", fontsize=10)
ax.set_yticklabels(['English', 'German', 'French', 'Spanish', 'Italian', 'Dutch'])

# create a list to collect the plt.patches data
totals = []
# find the ind. values and append to list
for i in ax.patches:
    totals.append(i.get_width())
# get total
total = sum(totals)

# set individual bar labels using above list
for i in ax.patches:
    ax.text(x=i.get_width(), y=i.get_y()+.35, 
            s=str(round((i.get_width()/total)*100, 2))+'%', 
            fontsize=10, color='black')

# invert for largest on top 
ax.invert_yaxis()

In [ ]:
# splitting the dataframes in language related sub-dataframes
df_eng = df[(df['language']=='en')]
df_de  = df[(df['language']=='de')]
df_fr  = df[(df['language']=='fr')]

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 3. Visualizing the Data with WordClouds
<a id='3. Visualizing the Data with WordClouds'></a>

**Preparing Steps**

In [ ]:
# import necessary libraries
from nltk.corpus import stopwords
from wordcloud import WordCloud
from collections import Counter
from PIL import Image

import re
import string

In [ ]:
# wrap the plotting in a function for easier access
def plot_wordcloud(wordcloud, language):
    plt.figure(figsize=(12, 10))
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis("off")
    plt.title(language + ' Comments\n', fontsize=18, fontweight='bold')
    plt.show()

**English WordCloud**

In [ ]:
wordcloud = WordCloud(max_font_size=None, max_words=200, background_color="lightgrey", 
                      width=3000, height=2000,
                      stopwords=stopwords.words('english')).generate(str(df_eng.comments.values))

plot_wordcloud(wordcloud, 'English')

**German WordCloud**

In [ ]:
wordcloud = WordCloud(max_font_size=None, max_words=150, background_color="powderblue",
                      width=3000, height=2000,
                      stopwords=stopwords.words('german')).generate(str(df_de.comments.values))

plot_wordcloud(wordcloud, 'German')

**French WordCloud**

In [ ]:
wordcloud = WordCloud(max_font_size=200, max_words=150, background_color="lightgoldenrodyellow",
                      #width=1600, height=800,
                      width=3000, height=2000,
                      stopwords=stopwords.words('french')).generate(str(df_fr.comments.values))

plot_wordcloud(wordcloud, 'French')

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 4. Sentiment Analysis
<a id='4. Sentiment Analysis'></a>

#### 4.1. Get used to VADER package
<a id='4.1. Get used to VADER package'></a>

An excellent and easy-to-read overview of sentiment analysis and the VADER package can be found on Jodie Burchell's <a href='http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html'> blogpost</a>. (I don't want to repeat what she said, I'd recommend reading it in her own words.)

In [ ]:
# load the SentimentIntensityAnalyser object in
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
# assign it to another name to make it easier to use
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# use the polarity_scores() method to get the sentiment metrics
def print_sentiment_scores(sentence):
    snt = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))

VADER belongs to a type of sentiment analysis that is based on lexicons of sentiment-related words. In this approach, each of the words in the lexicon is rated as positive or negative, and in many cases, **how** positive or negative. <br> Let's play around a bit and get familiar with this package:

In [ ]:
print_sentiment_scores("This raspberry cake is good.")

VADER produces four sentiment metrics from these word ratings, which you can see above. The first three - positive, neutral and negative - represent the proportion of the text that falls into those categories. As you can see, our example sentence was rated as 42% positive, 58% neutral, and 0% negative. 

The final metric, **the compound score**, is the sum of all of the lexicon ratings which have been standardised to range between -1 and 1. In this case, our example sentence has a rating of 0.44, which is pretty neutral.

In [ ]:
print_sentiment_scores("This raspberry cake is good.")
print_sentiment_scores("This raspberry cake is GOOD!")
print_sentiment_scores("This raspberry cake is VERY GOOD!!")
print_sentiment_scores("This raspberry cake is really GOOD! But the coffee is dreadful.")

In [ ]:
# getting only the negative score
def negative_score(text):
    negative_value = analyzer.polarity_scores(text)['neg']
    return negative_value

# getting only the neutral score
def neutral_score(text):
    neutral_value = analyzer.polarity_scores(text)['neu']
    return neutral_value

# getting only the positive score
def positive_score(text):
    positive_value = analyzer.polarity_scores(text)['pos']
    return positive_value

# getting only the compound score
def compound_score(text):
    compound_value = analyzer.polarity_scores(text)['compound']
    return compound_value

In [ ]:
negative_score("The food is really GOOD! But the service is dreadful.")

In [ ]:
neutral_score("The food is really GOOD! But the service is dreadful.")

In [ ]:
positive_score("The food is really GOOD! But the service is dreadful.")

In [ ]:
compound_score("The food is really GOOD! But the service is dreadful.")

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 4.2. Calculating Sentiment Scores
<a id='4.2. Calculating Sentiment Scores'></a>

Let's now have VADER produce all four scores for each of our English-language comments. As this takes roughly a quarter of an hour, it's a good idea to save the dataframe.

In [ ]:
%%time

df_eng['sentiment_neg'] = df_eng['comments'].apply(negative_score)
df_eng['sentiment_neu'] = df_eng['comments'].apply(neutral_score)
df_eng['sentiment_pos'] = df_eng['comments'].apply(positive_score)
df_eng['sentiment_compound'] = df_eng['comments'].apply(compound_score)

In [ ]:
# write the dataframe to a csv file in order to avoid the long runtime
#df_eng.to_csv('data/sentimentData/sentiment_df_eng', index=False)
#df = pd.read_csv('data/sentimentData/sentiment_df_eng')
df = df_eng

In [ ]:
df.head(2)

Let's investigate the distribution of all scores:

In [ ]:
# all scores in 4 histograms
fig, axes = plt.subplots(2, 2, figsize=(10,8))

# plot all 4 histograms
df.hist('sentiment_neg', bins=25, ax=axes[0,0], color='lightcoral', alpha=0.6)
axes[0,0].set_title('Negative Sentiment Score')
df.hist('sentiment_neu', bins=25, ax=axes[0,1], color='lightsteelblue', alpha=0.6)
axes[0,1].set_title('Neutral Sentiment Score')
df.hist('sentiment_pos', bins=25, ax=axes[1,0], color='chartreuse', alpha=0.6)
axes[1,0].set_title('Positive Sentiment Score')
df.hist('sentiment_compound', bins=25, ax=axes[1,1], color='navajowhite', alpha=0.6)
axes[1,1].set_title('Compound')

# plot common x- and y-label
fig.text(0.5, 0.04, 'Sentiment Scores',  fontweight='bold', ha='center')
fig.text(0.04, 0.5, 'Number of Reviews', fontweight='bold', va='center', rotation='vertical')

# plot title
plt.suptitle('Sentiment Analysis of Airbnb Reviews for Berlin\n\n', fontsize=12, fontweight='bold');

Remember what we said earlier: VADER produces four sentiment metrics from these word ratings (...). The first three - positive, neutral and negative - represent the **proportion of the text** that falls into those categories. (...). The final metric, **the compound score**, is the sum of all of the lexicon ratings which have been standardized to range between -1 and 1.

Finally, let’s use the method described to generate descriptive statistics that summarize the central tendency and dispersion of our dataset's compound score:

In [ ]:
percentiles = df.sentiment_compound.describe(percentiles=[.05, .1, .2, .3, .4, .5, .6, .7, .8, .9])
percentiles

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 4.3. Comparing Negative and Positive Comments
<a id='4.3. Comparing Negative and Positive Comments'></a>

In [ ]:
# assign the data
neg = percentiles['5%']
mid = percentiles['20%']
pos = percentiles['max']
names = ['Negative Comments', 'Okayish Comments','Positive Comments']
size = [neg, mid, pos]

# call a pie chart
plt.pie(size, labels=names, colors=['lightcoral', 'lightsteelblue', 'chartreuse'], 
        autopct='%.2f%%', pctdistance=0.8,
        wedgeprops={'linewidth':7, 'edgecolor':'white' })

# create circle for the center of the plot to make the pie look like a donut
my_circle = plt.Circle((0,0), 0.6, color='white')

# plot the donut chart
fig = plt.gcf()
fig.set_size_inches(7,7)
fig.gca().add_artist(my_circle)
plt.show()

Clearly, the bulk of the reviews are tremendously positive. Wouldn't it be interesting to know what the negative and positive comments are about? Let's have a look:

In [ ]:
# full dataframe with POSITIVE comments
df_pos = df.loc[df.sentiment_compound >= 0.95]

# only corpus of POSITIVE comments
pos_comments = df_pos['comments'].tolist()

In [ ]:
# full dataframe with NEGATIVE comments
df_neg = df.loc[df.sentiment_compound < 0.0]

# only corpus of NEGATIVE comments
neg_comments = df_neg['comments'].tolist()

Let's compare the length of both positive and negative comments:

In [ ]:
df_pos['text_length'] = df_pos['comments'].apply(len)
df_neg['text_length'] = df_neg['comments'].apply(len)

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(8,5))

sns.distplot(df_pos['text_length'], kde=True, bins=50, color='chartreuse')
sns.distplot(df_neg['text_length'], kde=True, bins=50, color='lightcoral')

plt.title('\nDistribution Plot for Length of Comments\n')
plt.legend(['Positive Comments', 'Negative Comments'])
plt.xlabel('\nText Length')
plt.ylabel('Percentage of Comments\n');

The mode for the text length of positive comments can be found more to the right than for the negative comments, which means most of the positive comments are longer than most of the negative comments. But the tail for negative comments is thicker.

In [ ]:
# read some positive comments
pos_comments[10:15]

In [ ]:
# read some negative comments
neg_comments[10:15]

Let's quickly check if a scatter plot may reveal some differences in the comments' sentiment with respect to the districts:

In [ ]:
sns.set_style("white")
cmap = sns.cubehelix_palette(rot=-.4, as_cmap=True)
fig, ax = plt.subplots(figsize=(11,7))

ax = sns.scatterplot(x="longitude", y="latitude", size='number_of_reviews', sizes=(5, 200),
                     hue='sentiment_compound', palette=cmap,  data=df)
ax.legend(bbox_to_anchor=(1.3, 1), borderaxespad=0.)
plt.title('\nAccommodations in Berlin by Number of Reviwws & Sentiment\n', fontsize=12, fontweight='bold')

sns.despine(ax=ax, top=True, right=True, left=True, bottom=True);

Not really...

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 4.4. Investigating Positive Comments
<a id='4.4. Investigating Positive Comments'></a>

**WordCloud**

After reading some of these reviews to get a feeling for what visitors applaud or complain about, WordClouds are a great tool to help us peek behind the curtain:

In [ ]:
wordcloud = WordCloud(max_font_size=200, max_words=200, background_color="palegreen",
                      width= 3000, height = 2000,
                      stopwords = stopwords.words('english')).generate(str(df_pos.comments.values))

plot_wordcloud(wordcloud, '\nPositively Tuned')

**Frequency Distribution**

Another method for visually exploring text is with frequency distributions. In the context of a text corpus, such a distribution tells us the prevalence of certain words. Here we use the Yellowbrick library.

In [ ]:
# importing libraries
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text.freqdist import FreqDistVisualizer
from yellowbrick.style import set_palette

In [ ]:
# vectorizing text
vectorizer = CountVectorizer(stop_words='english')
docs = vectorizer.fit_transform(pos_comments)
features = vectorizer.get_feature_names()

# preparing the plot
set_palette('pastel')
plt.figure(figsize=(18,8))
plt.title('The Top 30 most frequent words used in POSITIVE comments\n', fontweight='bold')

# instantiating and fitting the FreqDistVisualizer, plotting the top 30 most frequent terms
visualizer = FreqDistVisualizer(features=features, n=30)
visualizer.fit(docs)
visualizer.poof;

**Topic Modelling**

Next we'll explore topic modeling, an unsupervised machine learning technique for abstracting topics from collections of documents or, in our case, for identifying which topic is being discussed in a comment. 

Methods for topic modeling have evolved significantly over the last decade. In this section, we'll explore a technique called *Latent Dirichlet Allocation (LDA)*, a widely used topic modelling technique.

*1. Cleaning and Preprocessing*

In [ ]:
# importing libraries
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [ ]:
# prepare the preprocessing
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [ ]:
# removing stopwords, punctuations and normalizing the corpus
def clean(doc):
    stop_free = " ".join([word for word in doc.lower().split() if word not in stop])
    punc_free = "".join(token for token in stop_free if token not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(comment).split() for comment in pos_comments]

*2. LDA the Gensim way*

First, we create a Gensim dictionary from the normalized data, then we convert this to a bag-of-words corpus, and save both dictionary and corpus for future use.

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(doc_clean)
corpus = [dictionary.doc2bow(text) for text in doc_clean]

import pickle 
# uncomment the code if working locally
#pickle.dump(corpus, open('data/sentimentData/corpus.pkl', 'wb'))
#dictionary.save('data/sentimentData/dictionary.gensim')

In [ ]:
import gensim

# let LDA find 3 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)

# uncomment the code if working locally
#ldamodel.save('../input/sentimentData/model3.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

- The first topic includes words like *apartment*, *great*, and *walk*, and *minute*. This sounds like a topic related to convenient distances from the accommodation to wherever something interesting was to go to.
- The second topic includes words like *place*, *room*, *even*, and a mysterious *u* (perhaps u-bahn for the underground?). It seems unclear to me what this was supposed to be about. 
- And the third topic combines words like *great*, *place*, *stay*, and *apartment*, which sounds like a cluster related to overall satisfaction with the home.

In [ ]:
# now let LDA find 5 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

# uncomment the code if working locally
#ldamodel.save('../input/sentimentData/model5.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
# and finally 10 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

# uncomment the code if working locally
#ldamodel.save('../input/sentimentData/model10.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Putting it all together - the WordCloud, the Frequency Distribution and the Topic Modelling - it is often the following criteria that make someone rate an apartment **positively:**
1. **The apartment is clean, the bathroom is clean, the bed is comfortable.**
2. **The apartment is quiet and conducive to getting sound sleep.**
3. **The area is centrally located with short walking distances, good public transport connections, and has cafes and restaurants nearby.**

Apparently, getting the last two means trying to square the circle... but this is true for tourists all over the world.

Before we move on to the negative comments, let's visualize the LDA model:

*3. Visualizing topics*

The pyLDAvis library is designed to provide a visual interface for interpreting the topics derived from a topic model by extracting information from a fitted LDA topic model.

***The following code should be run locally only!***

In [ ]:
#dictionary = gensim.corpora.Dictionary.load('../input/sentimentData/dictionary.gensim')
#corpus = pickle.load(open('../input/sentimentData/corpus.pkl', 'rb'))

#import pyLDAvis.gensim

In [ ]:
# visualizing 5 topics
#lda = gensim.models.ldamodel.LdaModel.load('../input/sentimentData/model5.gensim')
#lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
#pyLDAvis.display(lda_display)

In short, the interface provides:

- a left panel that depicts a global view of the model (how prevalent each topic is and how topics relate to each other);
- a right panel containing a bar chart – the bars represent the terms that are most useful in interpreting the topic currently selected (what the meaning of each topic is).

On the left, the topics are plotted as circles, whose centers are defined by the computed distance between topics (projected into 2 dimensions). The prevalence of each topic is indicated by the circle’s area. On the right, two juxtaposed bars show the topic-specific frequency of each term (in red) and the corpus-wide frequency (in blueish gray). When no topic is selected, the right panel displays the top 30 most salient terms for the dataset.

In [ ]:
# visualizing 3 topics
#lda = gensim.models.ldamodel.LdaModel.load('../input/sentimentData/model3.gensim')
#lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
#pyLDAvis.display(lda_display)

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 4.5. Investigating Negative Comments
<a id='4.5. Investigating Negative Comments'></a>

**WordCloud**

In [ ]:
wordcloud = WordCloud(max_font_size=200, max_words=200, background_color="mistyrose",
                      width=3000, height=2000,
                      stopwords=stopwords.words('english')).generate(str(df_neg.comments.values))

plot_wordcloud(wordcloud, '\nNegatively Tuned')

**Frequency Distribution**

In [ ]:
# vectorizing text
vectorizer = CountVectorizer(stop_words='english')
docs = vectorizer.fit_transform(neg_comments)
features = vectorizer.get_feature_names()

# preparing the plot
set_palette('flatui')
plt.figure(figsize=(18,8))
plt.title('The Top 30 most frequent words used in NEGATIVE comments\n', fontweight='bold')

# instantiating and fitting the FreqDistVisualizer, plotting the top 30 most frequent terms
visualizer = FreqDistVisualizer(features=features, n=30)
visualizer.fit(docs)
visualizer.poof;

**Topic Modelling**

*1. Cleaning and Preprocessing*

In [ ]:
# calling the cleaning function we defined earlier
doc_clean = [clean(comment).split() for comment in neg_comments]

*2. LDA the Gensim way*

In [ ]:
# create a dictionary from the normalized data, convert this to a bag-of-words corpus
dictionary = corpora.Dictionary(doc_clean)
corpus = [dictionary.doc2bow(text) for text in doc_clean]

# save for later use
# uncomment the code if working locally
#pickle.dump(corpus, open('data/sentimentData/corpus_neg.pkl', 'wb'))
#dictionary.save('data/sentimentData/dictionary_neg.gensim')

In [ ]:
# let LDA find 3 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)

# uncomment the code if working locally
#ldamodel.save('../input/sentimentData/model3_neg.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
# now let LDA find 5 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

# uncomment the code if working locally
#ldamodel.save('../input/sentimentData/model5_neg.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
# and finally 10 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

# uncomment the code if working locally
#ldamodel.save('../input/sentimentData/model10_neg.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Once again, let's put all of the visualizations together and summarize what makes someone rate an apartment **negatively:**
1. **The apartment and/or bathroom (especially the shower) are dirty.**
2. **Problems in communicating with the host, e.g. one-sided cancellations by the host or to not being able to get a hold of him/her when having issues.**
3. **The area is too far away from public transport connections or doesn't meet vistors' expectations in some way.**

Before we finish analyzing the negative comments, let's visualize the LDA model:

*3. Visualizing topics*

***The following code should be run locally only!***

In [ ]:
#dictionary = gensim.corpora.Dictionary.load('./input/sentimentData/dictionary_neg.gensim')
#corpus = pickle.load(open('./input/sentimentData/corpus_neg.pkl', 'rb'))

In [ ]:
# visualizing 5 topics
#lda = gensim.models.ldamodel.LdaModel.load('../input/sentimentData/model5_neg.gensim')
#lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
#pyLDAvis.display(lda_display)

In [ ]:
# visualizing 3 topics
#lda = gensim.models.ldamodel.LdaModel.load('../input/sentimentDatamodel3_neg.gensim')
#lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
#pyLDAvis.display(lda_display)

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 5. Appendix 
<a id='5. Appendix'></a>

All resources used in this notebook are listed below.

Data
- Inside Airbnb: http://insideairbnb.com/get-the-data.html

WordClouds
- https://vprusso.github.io/blog/2018/natural-language-processing-python-3/
- https://www.datacamp.com/community/tutorials/wordcloud-python

Bar Charts
- http://robertmitchellv.com/blog-bar-chart-annotations-pandas-mpl.html

YellowBrick Visualization
- http://www.scikit-yb.org/en/latest/index.html

Language Detection
- TextBlob:
    - https://www.analyticsvidhya.com/blog/2018/02/natural-language-processing-for-beginners-using-textblob/
    - https://github.com/shubhamjn1/TextBlob/blob/master/Textblob.ipynb
    - https://stackoverflow.com/questions/43485469/apply-textblob-in-for-each-row-of-a-dataframe
    - https://textblob.readthedocs.io/en/dev/quickstart.html
<br>
- Spacy:
    - https://github.com/nickdavidhaynes/spacy-cld
    - https://spacy.io/usage/models
<br>
- Langdetect & LangId:
    - https://pypi.org/project/langdetect/ 
    - https://www.probytes.net/blog/python-language-detection/
    - https://github.com/hb20007/hands-on-nltk-tutorial/blob/master/8-1-The-langdetect-and-langid-Libraries.ipynb

Sentiment Analysis
- *"Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning"* (Paperback) by B. Bengfort, R. Bilbro, T. Ojeda, published by O′Reilly
- Jodie Burchell: http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
- Jodie Burchell: http://t-redactyl.io/blog/2017/01/how-do-we-feel-about-new-years-resolutions-according-to-sentiment-analysis.html
- Jodie Burchell: https://github.com/t-redactyl/Blog-posts/blob/master/2017-04-15-sentiment-analysis-in-vader-and-twitter-api.ipynb
- http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf

- Susan Li: https://towardsdatascience.com/latent-semantic-analysis-sentiment-classification-with-python-5f657346f6a3
- Sakshi Gupta (in R): https://towardsdatascience.com/uncovering-hidden-trends-in-airbnb-reviews-11eb924f2fec
- Dmytro Iakubovskyi: https://towardsdatascience.com/digging-into-airbnb-data-reviews-sentiments-superhosts-and-prices-prediction-part1-6c80ccb26c6a
- Dmytro Iakubovskyi: https://github.com/Dima806/Airbnb_project/blob/master/airbnb_final_analysis_v3.ipynb
- Maurizio Santamicone: https://medium.com/@mauriziosantamicone/seattle-confidential-unpacking-airbnb-reviews-with-sentiment-d421c15d8b8f
- Zhenyu: https://www.kaggle.com/zhenyufan/nlp-for-yelp-reviews/notebook?utm_medium=email&utm_source=intercom&utm_campaign=datanotes-2019

Topic Modeling / LDA
- Analytics Vidhya: https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
- Susan Li: https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
- https://radimrehurek.com/gensim/models/ldamodel.html
- https://www.objectorientedsubject.net/2018/08/experiments-on-topic-modeling-pyldavis/

Diverse
- https://data-viz-for-fun.com/2018/08/airbnb-data-viz/